In [1]:
%pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu129
%pip install roboflow split-folders matplotlib numpy pillow

Looking in indexes: https://download.pytorch.org/whl/nightly/cu129
  Using cached https://download.pytorch.org/whl/nightly/cu129/torch-2.9.0.dev20250905%2Bcu129-cp311-cp311-win_amd64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/nightly/cu129/torchvision-0.24.0.dev20250901%2Bcu129-cp311-cp311-win_amd64.whl.metadata (6.0 kB)
  Using cached https://download.pytorch.org/whl/nightly/cu129/torchaudio-2.8.0.dev20250901%2Bcu129-cp311-cp311-win_amd64.whl.metadata (7.4 kB)
   ---------------------------------------- 0.0/3.2 GB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 GB 3.7 MB/s eta 0:14:07
   ---------------------------------------- 0.0/3.2 GB 5.6 MB/s eta 0:09:27
   ---------------------------------------- 0.0/3.2 GB 6.2 MB/s eta 0:08:29
   ---------------------------------------- 0.0/3.2 GB 6.0 MB/s eta 0:08:45
   ---------------------------------------- 0.0/3.2 GB 5.7 MB/s eta 0:09:18
   ---------------------------------------- 0.0/3.2 GB 5.7 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sys
print(sys.executable)

C:\Users\acotj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
Name: roboflow
Version: 1.2.9
Summary: Official Python package for working with the Roboflow API
Home-page: https://github.com/roboflow-ai/roboflow-python
Author: Roboflow
Author-email: support@roboflow.com
License: UNKNOWN
Location: C:\Users\acotj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages
Requires: certifi, cycler, filetype, idna, kiwisolver, matplotlib, numpy, opencv-python-headless, pi-heif, Pillow, pillow-avif-plugin, python-dateutil, python-dotenv, PyYAML, requests, requests-toolbelt, six, tqdm, urllib3
Required-by: 


In [9]:
import os, time
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from roboflow import Roboflow
import splitfolders
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from PIL import Image
from collections import defaultdict


In [ ]:
## DONT RUN THIS WAS USED FOR DOWLOADING DATASET
# --- IGNORE ---
# rf = Roboflow(api_key="AWN4nVlJhx52ROuzjIPX")
# project = rf.workspace("itrash").project("apple-khthj")
# version = project.version(4)
#dataset = version.download("multiclass")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Apple-4 in multiclass:: 100%|██████████| 3383/3383 [00:05<00:00, 575.75it/s]


In [ ]:
## LOADING DATASET AND TRAINING THE MODEL

class AppleRipenessDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.class_names = list(self.annotations.columns[1:])  # skip filename col

        if len(self.annotations) == 0:
            print(f"[WARNING] No rows found in {csv_file}")

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = self.annotations.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"[ERROR] Image not found: {img_path}")

        image = Image.open(img_path).convert("RGB")
        labels = self.annotations.iloc[idx, 1:].values.astype("float")
        label = torch.tensor(labels).argmax().long()

        if self.transform:
            image = self.transform(image)

        return image, label


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

data_dir = "Apple-4"  # adjust if different

data_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

datasets_custom = {
    split: AppleRipenessDataset(
        csv_file=os.path.join(data_dir, split, "_classes.csv"),
        img_dir=os.path.join(data_dir, split),
        transform=data_transforms
    )
    for split in ["train", "valid", "test"]
}

dataloaders = {
    split: DataLoader(datasets_custom[split], batch_size=32, shuffle=True, num_workers=0)
    for split in ["train", "valid", "test"]
}

dataset_sizes = {x: len(datasets_custom[x]) for x in ["train", "valid", "test"]}
class_names = datasets_custom["train"].class_names
print("Classes:", class_names)
print("Dataset sizes:", dataset_sizes)

model = resnet18(weights=ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        for phase in ["train", "valid"]:
            if dataset_sizes[phase] == 0:
                print(f"[WARNING] No data in {phase} set. Skipping...")
                continue

            model.train() if phase == "train" else model.eval()
            running_loss, running_corrects = 0.0, 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == "train":
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == "valid" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s")
    print(f"Best val Acc: {best_acc:.4f}")

    model.load_state_dict(best_model_wts)
    return model


model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)
torch.save(model.state_dict(), "apple_resnet18.pth")
print("Model saved as apple_resnet18.pth")

model.eval()
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in dataloaders["test"]:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Using device: cuda:0
Classes: ['apple_ripe', 'apple_rotten', 'apple_unripe']
Dataset sizes: {'train': 3001, 'valid': 247, 'test': 127}
Epoch 1/10
----------
train Loss: 0.5715 Acc: 0.7517
valid Loss: 0.1590 Acc: 0.9474
Epoch 2/10
----------
train Loss: 0.3479 Acc: 0.8527
valid Loss: 0.1857 Acc: 0.9231
Epoch 3/10
----------
train Loss: 0.2453 Acc: 0.9034
valid Loss: 0.1330 Acc: 0.9555
Epoch 4/10
----------
train Loss: 0.1786 Acc: 0.9364
valid Loss: 0.1452 Acc: 0.9433
Epoch 5/10
----------
train Loss: 0.1143 Acc: 0.9660
valid Loss: 0.1900 Acc: 0.9312
Epoch 6/10
----------
train Loss: 0.0794 Acc: 0.9777
valid Loss: 0.1978 Acc: 0.9312
Epoch 7/10
----------
train Loss: 0.0518 Acc: 0.9910
valid Loss: 0.1837 Acc: 0.9352
Epoch 8/10
----------
train Loss: 0.0345 Acc: 0.9953
valid Loss: 0.1567 Acc: 0.9393
Epoch 9/10
----------
train Loss: 0.0295 Acc: 0.9980
valid Loss: 0.1884 Acc: 0.9312
Epoch 10/10
----------
train Loss: 0.0299 Acc: 0.9980
valid Loss: 0.1945 Acc: 0.9352
Training complete in 8m 

In [ ]:
## LOADING THE MODEL

model = resnet18(weights=None)  # no pretrained weights this time
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # 3 classes: ripe, rotten, unripe

model.load_state_dict(torch.load("apple_resnet18.pth", map_location=device))
model = model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
## EVALUATING MODEL WITH VALIDATION SET (UNSEEN DATA)

correct = 0
total = 0
class_correct = defaultdict(int)
class_total = defaultdict(int)

with torch.no_grad():
    for inputs, labels in dataloaders["valid"]:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (preds == labels).sum().item()

        for label, pred in zip(labels, preds):
            class_total[label.item()] += 1
            if label.item() == pred.item():
                class_correct[label.item()] += 1


val_acc = 100 * correct / total
print(f"\nValidation Accuracy (loaded model): {val_acc:.2f}%")

# per-class accuracy
for i, class_name in enumerate(class_names): 
    if class_total[i] > 0:
        acc = 100 * class_correct[i] / class_total[i]
        print(f"{class_name}: {acc:.2f}%")
    else:
        print(f"{class_name}: No samples in validation set")


Validation Accuracy (loaded model): 93.52%
apple_ripe: 92.86%
apple_rotten: 91.43%
apple_unripe: 100.00%
